## Different Approaches to Conversational Modeling
- 3.3.1
主要会学习分层RNN 结构 - HRED（Hierarchical Recurrent Encoder-Decoder），能同时对句子和对话语境（上下文）进行建模，来实现多轮对话。
- 3.3.2
任务型对话和encoder-decoder模型结合的各种方法介绍。
- 3.3.3
强化学习方法在此领域的应用。
- 3.3.4
介绍了其他非标准的seq2seq的模型

### 3.3.1 HRED

Building end-to-end dialogue systems using generative hierarchical neural network models 论文提出了一种 分层 RNN 结构 - HRED（Hierarchical Recurrent Encoder-Decoder），能同时对句子和对话语境（上下文）进行建模，来实现多轮对话。

如果不使用分层RNN，在传统Seq2Seq模型基础上，如果我们想得到context 信息应该怎么做。
第一个想法是将上一个句子的 final state 作为下一个句子的 initial state，然后将句子信息不断传递下去，这样的话 context vector 里的信息会在 propagation 的过程中会被新句子的词语逐步稀释，对信息/梯度的传播极不友好。

因此为了让信息更好的传递，我们可能会考虑把 final state 传递到下一个句子的 last state，而不是 initial state，然后用拼接或者非线性的方法来表达之前的和当前的句子信息。

直接将语境中的多个 utterance vector 提取出来再用一个 RNN 来处理，捕捉 context 信息，这就有了分层 RNN。


![HRED](./data/HRED.png)

HRED 在传统 encoder-decoder 模型上，额外增加了一个 encoder，相比于普通的 RNN-LM 来说，考虑了 turn-taking nature，能够对上下文进行建模，减少了相邻句子间的计算步骤，有助于信息/梯度的传播，从而实现多轮对话。整个过程有下面三个阶段：

- encoder RNN:

第一个 encoder 和标准的 seq2seq 相同，将一句话编码到固定长度的 utterance vector，也就是 RNN 的 last hidden state
encoder RNN 或者说 utterance RNN 记忆的是对话的细节

- context RNN:

个句子的 utterance vector 作为第二个 encoder 也就是 context-level encoder 各个时间上的的输入，对应长度为 n 的 sequence，产生一个 context vector 实现对语境的编码，也就是 RNN 的 output (注意这里不是 last hidden state)
context RNN 记忆的是更为全局的语义信息

- decoder RNN

上一个句子的 utterance vector 作为 response 的初始状态，目前为止产生的 context vector 和上一个单词的 word embedding 拼接作为 decoder 的输入

然而 HRED 相对于传统的 Seq2Seq 模型的提高并不明显，bootstrapping 的作用更加明显。一方面可以用 pre-trained word embedding，另一方面可以使用其他 NLP 任务的数据预训练我们的模型，使得模型的参数预先学到一些对自然语言的理解，然后再来学习聊天任务。